In [21]:
!ls

bounding_box_data  data		   model		 testgrabcut.py
Coursera	   GrabCut.ipynb   PreprocessData.ipynb  UNet.ipynb
cpfromhome.sh	   grabcuttest.py  README.md		 unet_model.py


Import files and setup params

In [4]:
# settings
from keras.preprocessing.image import ImageDataGenerator
from model.unet_model import *

# dimensions of our images.
img_width, img_height = 256, 256
train_dir = 'data/train'
validation_dir = 'data/valid'
image_folder = 'images'
masks_folder = 'masks'
num_epochs = 50
batch_size = 8

Create image and mask generator to train

In [3]:
# we create two instances with the same arguments
data_gen_args = dict(rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
image_datagen = ImageDataGenerator(**data_gen_args)
masks_datagen = ImageDataGenerator(**data_gen_args)

test_datagen = ImageDataGenerator(rescale=1./255)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1

image_generator = image_datagen.flow_from_directory(
    train_dir,
    classes = [image_folder],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

mask_generator = masks_datagen.flow_from_directory(
    train_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

valid_image_generator = test_datagen.flow_from_directory(
    validation_dir,
    classes = [image_folder],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

valid_mask_generator = test_datagen.flow_from_directory(
    validation_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)
validation_generator = zip(valid_image_generator, valid_mask_generator)

Found 612 images belonging to 1 classes.
Found 612 images belonging to 1 classes.
Found 196 images belonging to 1 classes.
Found 196 images belonging to 1 classes.


In [6]:
model = unet()
model.compile(optimizer = Adam(lr = 1e-4), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/thienhn4/polypseg/model/unet_model.py:51: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


In [ ]:
model_checkpoint = ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5',
                                   monitor='loss',
                                   verbose=1, 
                                   save_best_only=True)

model.fit_generator(train_generator,
                    steps_per_epoch=612//batch_size,
                    epochs=num_epochs,
                    validation_data = validation_generator,
                    validation_steps = 196//batch_size,
                    callbacks=[model_checkpoint])

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
76/76 [==============================] - 8473s 111s/step - loss: 0.3195 - acc: 0.9081 - val_loss: 0.3682 - val_acc: 0.9546

Epoch 00001: loss improved from inf to 0.32092, saving model to weights.01-0.37.hdf5
Epoch 2/50
76/76 [==============================] - 7614s 100s/step - loss: 0.2515 - acc: 0.9072 - val_loss: 0.3518 - val_acc: 0.9561

Epoch 00002: loss improved from 0.32092 to 0.25181, saving model to weights.02-0.35.hdf5
Epoch 3/50
 2/76 [..............................] - ETA: 1:44:16 - loss: 0.2527 - acc: 0.8960